In [4]:
import pandas as pd

import json
import numpy as np
import datetime as dt

import matplotlib
import matplotlib.pyplot as plt

%matplotlib inline

import urllib3


In [5]:
dtype_train = {"id": np.int32, "keyword": "category", "target" : bool}
train = pd.read_csv("train.csv", dtype = dtype_train)

In [30]:
df_tweet_link = train[["text","target"]].copy()

In [35]:
def get_link(tweet):
    res =[]
    for w in tweet.split(' '):
        if ("http" in w) or ("https" in w):
            res.append(w)
    return (' ').join(res)

def len_link(links):
    if links=="":
        return 0
    return len(links.split(" "))

In [36]:
df_tweet_link['link'] = df_tweet_link['text'].apply(lambda x: get_link(x))

In [37]:
df_tweet_link['link_cant']= df_tweet_link['link'].apply(lambda x: len_link(x))

In [40]:
df_tweet_link = df_tweet_link[df_tweet_link.link_cant != 0]

In [56]:
text = []
file = open('url_tweets.txt', "r")
for l in file:
    text.append(l[:-1])
file.close

<function TextIOWrapper.close()>

In [98]:
df_tweet_link['url'] = pd.Series()

res = []
i = 0
b = 0
f = 0

for cant in df_tweet_link['link_cant']:
    links = []
    if(f+cant<1702):
        break
    f += cant
    links = text[i:f]
    b+= cant
    res.append((" ").join(links))

res

/home/anichu/Programs/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  """Entry point for launching an IPython kernel.
/home/anichu/Programs/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


[]

In [91]:
df_tweet_link.head(5)

,text,target,link,link_cant,url
31,@bbcmtd Wholesale Markets ablaze http://t.co/l...,True,http://t.co/lHYXEOHY6C,1,https://twitter.com/dannypurewal/status/629284...
32,We always try to bring the heavy. #metal #RT h...,False,http://t.co/YAo1e0xngw,1,https://twitter.com/dannypurewal/status/629284...
33,#AFRICANBAZE: Breaking news:Nigeria flag set a...,True,http://t.co/2nndBGwyEi,1,https://twitter.com/dannypurewal/status/629284...
35,On plus side LOOK AT THE SKY LAST NIGHT IT WAS...,False,http://t.co/qqsmshaJ3N,1,https://twitter.com/dannypurewal/status/629284...
37,INEC Office in Abia Set Ablaze - http://t.co/3...,True,http://t.co/3ImaomknnA,1,https://twitter.com/dannypurewal/status/629284...


In [9]:
# recibe un tweet y devuelve una cadena con los links a donde apuntas los shortened URL separados con una coma

def csv_of_shorturl(text):
    res = []
    servidor ='http://t.co' #esel servidor de twitter
    tweet = text.replace("\n",' ').replace(";",' ')
            
    for w in tweet.split(" "):
        if (servidor in w):
            res.append(w)
    return (" ").join(res)

In [5]:
df_tweet_link['links'] = df_tweet_link['text'].apply(csv_of_shorturl)

/home/anichu/Programs/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [6]:
def convert_url(short_url):
    res = ""
    for l in short_url.split(" "):
        try:
            res = urllib3.urlopen(l).urlgeturl()
            print(res)
        except:
            pass
    return res

In [7]:
df_tweet_link = df_tweet_link[df_tweet_link["links"]!=""]

In [8]:
short_url = df_tweet_link["links"][31]
short_url

'http://t.co/lHYXEOHY6C'

In [9]:
http = urllib3.PoolManager()

In [10]:
res = {}
def get_url(links):
    res = []
    url = ""
    for link in links.split(" "):
        try:
            res[link] = http.request('GET', link)
            print(res[link].data)
        except:
            url = ""
        res.append(url)
    return url

In [25]:
tot = 0
i = 0 
for w in df_tweet_link['links']:
    if (tot==1702):
        break
    tot += len(w.split(" "))
    i+=1

In [26]:
i

1416

In [12]:
#df_tweet_link['links'].apply(get_url)

In [13]:
# Guardo el dataframe porque se tarda mucho en calcularlo. Así, sólo lo consigo una vez.
#df_tweet_link.to_csv("df_tweet_link.csv") 